In [2]:
!pip install --upgrade azureml-sdk[notebooks,explain,automl,contrib]

     |████████████████████████████████| 24.0MB 43kB/s  eta 0:00:01    |█████████▌                      | 7.1MB 2.9MB/s eta 0:00:06     |████████████                    | 9.0MB 2.9MB/s eta 0:00:06     |██████████████▉                 | 11.1MB 2.9MB/s eta 0:00:05     |██████████████████████████      | 19.5MB 5.9MB/s eta 0:00:01     |██████████████████████████████▋ | 23.0MB 5.9MB/s eta 0:00:01
     |████████████████████████████████| 1.0MB 21.2MB/s eta 0:00:01
     |████████████████████████████████| 102kB 23.1MB/s ta 0:00:01
     |████████████████████████████████| 29.0MB 39kB/s eta 0:00:0112
     |████████████████████████████████| 112kB 24.4MB/s eta 0:00:01
     |████████████████████████████████| 5.3MB 32.1MB/s eta 0:00:01
     |████████████████████████████████| 204kB 17.5MB/s eta 0:00:01
     |████████████████████████████████| 71kB 18.6MB/s eta 0:00:01


     |████████████████████████████████| 1.8MB 29.4MB/s eta 0:00:01


     |████████████████████████████████| 101.4MB 11kB/s  eta 0:00:01��██████                    | 38.2MB 768kB/s eta 0:01:23kB/s eta 0:01:21████                  | 44.1MB 587kB/s eta 0:01:38            | 48.1MB 8.1MB/s eta 0:00:07     |██████████████████▍             | 58.1MB 15.1MB/s eta 0:00:03     |███████████████████▋            | 62.2MB 15.1MB/s eta 0:00:03     | 64.2MB 15.1MB/s eta 0:00:03     |█████████████████████           | 66.6MB 1.1MB/s eta 0:00:33     |██████████████████████          | 69.9MB 1.1MB/s eta 0:00:30�████████████▏      | 79.8MB 1.2MB/s eta 0:00:18kB/s eta 0:00:16��█████   | 91.6MB 764kB/s eta 0:00:13��██████████████████████████▏ | 95.6MB 764kB/s eta 0:00:08
     |████████████████████████████████| 51kB 13.6MB/s eta 0:00:01


ERROR: azureml-contrib-opendatasets 1.0.45 has requirement azureml-telemetry==1.0.45.*, but you'll have azureml-telemetry 1.0.55 which is incompatible.
  Found existing installation: azureml-core 1.0.48
    Uninstalling azureml-core-1.0.48:
      Successfully uninstalled azureml-core-1.0.48
  Found existing installation: azureml-pipeline-core 1.0.48
    Uninstalling azureml-pipeline-core-1.0.48:
      Successfully uninstalled azureml-pipeline-core-1.0.48
  Found existing installation: azureml-train-restclients-hyperdrive 1.0.48
    Uninstalling azureml-train-restclients-hyperdrive-1.0.48:
      Successfully uninstalled azureml-train-restclients-hyperdrive-1.0.48
  Found existing installation: azureml-telemetry 1.0.45
    Uninstalling azureml-telemetry-1.0.45:
      Successfully uninstalled azureml-telemetry-1.0.45
  Found existing installation: azureml-train-core 1.0.48
    Uninstalling azureml-train-core-1.0.48:
      Successfully uninstalled azureml-train-core-1.0.48
  Found existing

In [23]:
from azureml.core import Workspace, Experiment, Run
from azureml.core.authentication import InteractiveLoginAuthentication
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

import math, random, pickle

In [6]:
interactive_auth = InteractiveLoginAuthentication(tenant_id='22179471-b099-4504-bfdb-3f184cdae122')

ws = Workspace(subscription_id = '6e98728e-fd72-4501-9a4f-3eadca0fa6be', 
               resource_group = 'genisoft-ai', 
               workspace_name = 'aml-learning',
              auth=interactive_auth)

Performing interactive authentication. Please follow the instructions on the terminal.
To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code FRY436S9B to authenticate.
Interactive authentication successfully completed.


In [8]:
ws.write_config()

In [16]:
f = open('.azureml/config.json', 'r')
f.read()

'{"Id": null, "Scope": "/subscriptions/6e98728e-fd72-4501-9a4f-3eadca0fa6be/resourceGroups/genisoft-ai/providers/Microsoft.MachineLearningServices/workspaces/aml-learning"}'

## Create a compute target

In [21]:
# Choose a name for your CPU cluster
cpu_cluster_name = 'aml01'

# Verify that cluster does not exist already
try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',
                                                          max_nodes=3)
    cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

cpu_cluster.wait_for_completion(show_output=True)

Found existing cluster, use it.
Succeeded
AmlCompute wait for completion finished
Minimum number of nodes requested have been provisioned


In [24]:
experiment = Experiment(workspace = ws, name = "my-first-experiment")

In [26]:
run = experiment.start_logging()

pi_counter = 0
n_iter = 100000

# Log total number of iterations
run.log("Number of iterations",n_iter)

for i in range(1,n_iter):
    # Monte Carlo step to update estimate
    x = random.random()
    y = random.random()
    if x*x + y*y < 1.0:
        pi_counter += 1
    pi_estimate = 4.0*pi_counter / i
    
    # Log convergence every 10000 iterations
    if i%10000==0:
        error = math.pi-pi_estimate
        run.log("Pi estimate",pi_estimate)
        run.log("Error",error)

# Log final results
run.log("Final estimate: ",pi_estimate)
run.log("Final error: ",math.pi-pi_estimate)

# Write file containing pi value into run history
with open("pi_estimate.txt","wb") as f:
    pickle.dump(str(pi_estimate),f)
run.upload_file(name = 'outputs/pi_estimate.txt', path_or_stream = './pi_estimate.txt')

# Complete tracking and get link to details
run.complete()

In [28]:
ds = ws.get_default_datastore()
print(ds.datastore_type, ds.account_name, ds.container_name)

AzureBlob amllearning7418651507 azureml-blobstore-7f62eb3c-d151-4a13-ad81-878f70d97a3d
